In [1]:
import os
import sys
import time
import json
import numpy as np
import pandas as pd
import networkx as nx
from glob import glob
from tqdm import tqdm

import geopandas as gpd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, fsolve
from scipy.sparse import csr_array,csc_array,save_npz,load_npz

sys.path.append("/mnt/beegfs/lcesarini/ge-distance/src")

os.chdir(f"/mnt/beegfs/lcesarini/ge-distance/")
from graph_ensembles.sparse import MultiFitnessModel as MFM

In [6]:
PLOT=True
NODES=100
COEFF_EDGES=4
SLOW='sparse'
JOB_NAME='pippo'
n_grid=95

arr_ul=load_npz(f'src/distance/out/arr_input_ul_{n_grid}.npz')

sample_nodes=np.random.randint(0,arr_ul.shape[0],NODES)
arr_ul=arr_ul[sample_nodes]


N_VERT=arr_ul.shape[0]
N_EDGE=arr_ul.shape[0] * COEFF_EDGES
N_SECT= np.unique(arr_ul.toarray()[:,1]).shape[0]
PROP_IN=np.random.randint(1,6,N_VERT * N_SECT).reshape(-1,N_SECT)
PROP_OUT=np.random.randint(0,5,N_VERT * N_SECT).reshape(-1,N_SECT)

arr_ul.toarray()

array([[ 651838,      13,      87],
       [3327329,      25,      27],
       [3982254,      27,      20],
       [1514525,      31,      73],
       [1236380,      27,      62],
       [4688263,      13,      91],
       [3475082,      19,      27],
       [2012312,       9,      59],
       [4130131,      13,      11],
       [4661482,      35,      13],
       [1452188,      17,      72],
       [1667330,       8,      69],
       [ 438035,       9,      79],
       [3509011,      11,      26],
       [ 978909,      30,      76],
       [1322294,       9,      76],
       [3364501,      27,      27],
       [2483615,      27,      55],
       [2750137,      19,      47],
       [1743493,      11,      69],
       [3165271,      30,      30],
       [4271574,      13,       7],
       [3104103,      13,      47],
       [3628841,      19,      32],
       [4161140,      38,      11],
       [1518781,      23,      73],
       [1385636,       4,      62],
       [2003641,      13,   

In [5]:
arr_ul.max(axis=0).toarray()

array([[4741993,      39,     909]], dtype=int32)

In [3]:
gmfm=MFM(
            num_vertices=N_VERT,
            num_edges=N_EDGE,
            num_labels=N_SECT,
            prop_out=PROP_OUT,
            prop_in=PROP_IN,
            param=1,
            id_grid=arr_ul[:,2].toarray(),
            n_ul=95
            # param=np.random.randint(1,10,N_SECT),
            # num_edges_label=np.random.randint(2,5,N_SECT),
            # selfloops=1,
            # per_label=1,
    )

0


In [4]:
gmfm.n_ul

95

In [6]:
test_i=np.random.randint(0,N_VERT)
test_j=np.random.randint(0,N_VERT)

if test_i==test_j:
    test_j+=1

print(f"Distance {test_i},{test_j}: {gmfm.prop_dyad(test_i,test_j,gmfm.id_grid,gmfm.dist_mat):.2f}")
print("post computation distance")
# print(gmfm.prop_dyad(1,2,dist_mat.toarray())))
gmfm.fit(verbose=False)

# dir(gmfm.solver_output)

# print(gmfm.solver_output.x)
# gmfm.solver_output.target
# gmfm.solver_output.f_seq

NameError: name 'gmfm' is not defined

In [16]:
print(f"\n---------------------\n")
print(f"\n---------------------\n")
print(f"exp_av_nn_prop:{gmfm.exp_av_nn_prop()}")
# print(f"expected_out_degree:{gmfm.expected_out_degree()}")
print(f"\n---------------------\n")
# print(f"expected_out_degree_by_label:{gmfm.expected_out_degree_by_label()}")
print(f"expected_in_degree:{gmfm.expected_in_degree()}")
# print(f"expected_in_degree_by_label:{gmfm.expected_in_degree_by_label()}")
print(f"expected_num_edges:{gmfm.expected_num_edges()}")
print(f"\n---------------------\n")
# print(f"expected_num_edges_label:{gmfm.expected_num_edges_label()}")

# for ele in dir(gmfm):
#     if "__" not in ele:
#         print(ele) 

x=gmfm.sample()
sampled_nx=x.to_networkx()



---------------------


---------------------



TypeError: not enough arguments: expected 8, got 0

In [19]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap
gif_filename = f'src/distance/tmp/graph_sampled_{N_VERT}_{N_EDGE}_{JOB_NAME}.gif'

with open(f'src/distance/out/UL_italy_poly_assigned_{n_grid}.json', 'r', encoding='utf-8') as f:
    r=json.load(f) 

gdf=json.loads(r)
lon=[]
lat=[]
for f in (gdf['features']):
    lon.append(f['properties']['lng'])
    lat.append(f['properties']['lat'])

# lon=lon[:NODES]
# lat=lat[:NODES]
lon=np.array(lon)[sample_nodes]
lat=np.array(lat)[sample_nodes]

for f in glob(f"src/distance/tmp/*_{N_VERT}_{N_EDGE}_{JOB_NAME}.png"):
    os.remove(f)

if os.path.exists(gif_filename):
    os.remove(gif_filename)
else:
    print("The file does not exist")

# dict_pos={x:[np.random.uniform(10,15),np.random.uniform(40,45)] for i,x in enumerate(sampled_nx.nodes)}
for _ in range(10):

    SAMPLED_NX=gmfm.sample().to_networkx()
    fig,ax=plt.subplots(1,1,figsize=(16,16))
    m = Basemap(
            projection='merc',
            llcrnrlon=5,#np.min(lon)*0.995,
            llcrnrlat=35,#np.min(lat)*0.9995,
            urcrnrlon=20,#np.max(lon)*1.005,
            urcrnrlat=47.2,#np.max(lat)*1.0005,
            lat_ts=0,
            resolution='l',
            ax=ax,
            suppress_ticks=False)
    # m.etopo()
    m.drawcountries(linewidth = 3);
    m.drawstates(linewidth = 0.2)
    m.drawcoastlines(linewidth=3)
    m.fillcontinents(color='coral',lake_color='aqua')
    m.drawmapboundary(fill_color='aqua') 
    dict_pos={x:list(m(lon[i],lat[i])) for i,x in enumerate(SAMPLED_NX.nodes)}

    # nx.draw(SAMPLED_NX,
    #         pos=dict_pos,
    #         alpha=0.5,
    #         edge_color='green',
    #         node_color='red',node_size=30,
    #         ax=ax,with_labels=True);
    
    nx.draw_networkx_nodes(G = SAMPLED_NX, pos = dict_pos, nodelist = SAMPLED_NX.nodes(), 
                    node_color = 'r', alpha = 0.8, node_size = 100, label=True)
    nx.draw_networkx_edges(G = SAMPLED_NX, pos = dict_pos, edge_color='g',
                            alpha=0.2, arrows = True)
    plt.tight_layout()
    plt.savefig(f"src/distance/tmp/sample_{_}_{N_VERT}_{N_EDGE}_{JOB_NAME}.png", dpi = 300)
    plt.close()


#CREATE the GIF
# Directory containing PNG images
png_dir = 'src/distance/tmp/'
list_png=glob(f"{png_dir}/sample*_{N_VERT}_{N_EDGE}_{JOB_NAME}.png")
# List PNG files in the directory
list_png.sort()
# Create GIF filename

frames=[]
for img in list_png[:10]:
    frames.append(imageio.imread(img))
imageio.mimsave(gif_filename, frames, 'GIF', duration=250)

print("GIF created successfully.")

The file does not exist
GIF created successfully.
